# **Tutorial 6**

### **Dask Pipeline Executor Advanced**

In this tutorial, the objetive is present some specific details of the `DaskExecutorPipeline` and how you can configure everything that is related to this executor type.

First, all the previous tutorials are creating an instance locally. This is the easiest way to test a Dask pipeline. Indeed, this is not the common practice when you are dealing with cluster in some cloud environment or HPC machines. Usually, developers persist data to split a huge data accross nodes/workers. So, let's see below how you can connect to a running Dask Cluster using our executor.

In [ ]:
from dasf.pipeline.executors import DaskPipelineExecutor

# machine 1.1.1.1: dask-scheduler
# machine 2.2.2.2: dask-worker/dask-cuda-worker 1.1.1.1
# machine 3.3.3.3: dask-worker/dask-cuda-worker 1.1.1.1
# machine 4.4.4.4: dask-worker/dask-cuda-worker 1.1.1.1
# machine 5.5.5.5: dask-worker/dask-cuda-worker 1.1.1.1
dask = DaskPipelineExecutor(address="1.1.1.1", port=8687)

You should not be concerned if the target supports GPUs or not. Our executor automatically discovers what is the cluster type. On the other hand, Dasf does not support heterogeneous configurations like workers with GPUs and workers without GPUs simultaneously.

The second example is now related to how you can limit a local Dask executor. When you provision a cloud environment with Dask workers, you already configure the resources limits. So, when you connect into it, you just need to run a pipeline. It is different when you initiate a local cluster. It automatically sets the maximum limits according the machine free resources. The example below is a local cluster with some customizations.

In [ ]:
dask = DaskPipelineExecutor(local=True, n_workers=4, threads_per_worker=1, processes=False,
                            memory_limit='3GB', silence_logs=False)

The last option you can set using a Dask executor is the memory profiler. After you executing a Dask data graph, you can see the memory peak of each worker no matter if they are local or remote.

You should be careful because if you run your pipeline multiple times, Dask cannot free memory properly and it can report more memory usage than it really was.

In [ ]:
dask = DaskPipelineExecutor(profiler="memusage")

Another example to start a executor is using a PBS executor if you are running your code inside a cluster who manages jobs.

In [ ]:
from dasf.pipeline.executors import DaskPBSPipelineExecutor

pbs = DaskPBSPipelineExecutor(cores=4, memory='20GB', processes=4, queue='myqueue', walltime='02:00:00', shebang='#!/bin/bash', n_workers=4)

The framework put this cluster requirement into the job queue, the code will wait until PBS executes the worker(s) and once it is ready you can run your code normally. This approach has only one problem, you cannot kick two jobs for different queues to use heterogeneous machines for instance. But this is the way PBS was implemented also.